In [5]:
# Chapter 8 — ICS Evaluation and Bounded Optimization
# Purpose:
# Evaluate the trained ICS from Chapter 7 as a human governed Advisory Assistant.
# The ICS predicts reactor configuration outputs and provides uncertainty aware safety gating.
# It can also support bounded optimization by ranking candidate designs within fixed envelopes.
# Inputs (upload to /content):
# - augmented_dataset.parquet (from Chapter 6)
# - xgb_keff_est.joblib
# - xgb_flux_shape_metric.joblib
# - xgb_breeding_proxy_norm.joblib
# - xgb_total_power_W.joblib
# - xgb_avg_core_power_density_W_cm3.joblib
# - xgb_conformal_residual_quantiles.json
# - (optional) chapter7_training_report.json
# Outputs (saved under Chapter8_Output/):
# - chapter8_scenarios.json
# - chapter8_scenario_predictions.csv
# - chapter8_candidate_rankings.csv
# - chapter8_coverage_report.json
# - chapter8_figures/*.png

import os
import json
import math
import random
from dataclasses import dataclass
from typing import Dict, Any, Tuple, List, Optional

import numpy as np
import pandas as pd
import joblib

from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from scipy.stats import wasserstein_distance, ks_2samp

import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler

# Block 1 — Reproducibility and Paths defining
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

OUT_DIR = "Chapter8_Output"
FIG_DIR = os.path.join(OUT_DIR, "chapter8_figures")
os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs(FIG_DIR, exist_ok=True)

AUG_PATH = "/content/augmented_dataset.parquet"

XGB_MODEL_FILES = {
    "keff_est": "/content/xgb_keff_est.joblib",
    "flux_shape_metric": "/content/xgb_flux_shape_metric.joblib",
    "breeding_proxy_norm": "/content/xgb_breeding_proxy_norm.joblib",
    "total_power_W": "/content/xgb_total_power_W.joblib",
    "avg_core_power_density_W_cm3": "/content/xgb_avg_core_power_density_W_cm3.joblib",
}

CONFORMAL_PATH = "/content/xgb_conformal_residual_quantiles.json"
CH7_REPORT_PATH = "/content/chapter7_training_report.json"  # optional

# Block 2 — Column Definitions
X_COLS = [
    "core_radius_cm",
    "blanket_thickness_cm",
    "height_cm",
    "initial_U233_frac",
]

Y_COLS = [
    "keff_est",
    "flux_shape_metric",
    "breeding_proxy_norm",
    "total_power_W",
    "avg_core_power_density_W_cm3",
]

LOG_TARGETS = {
    "breeding_proxy_norm",
    "total_power_W",
    "avg_core_power_density_W_cm3",
}

# In Chapter 7 code, conformal residual quantiles are computed in physical space.
# Therefore, we apply +/- q in physical space after inverse-transforming predictions.

# Block 3 — Human-defined Safety Envelopes (from Chapters 5 and 6)
KEFF_MAX = 0.98
POWER_MAX_W = 5e8
PD_MAX_W_CM3 = 100.0

# Block 4 — The Helpers
def load_json(path: str) -> Dict[str, Any]:
    if not os.path.exists(path):
        return {}
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)


def inverse_transform_y(y_t: np.ndarray, target: str) -> np.ndarray:
    """Inverse transform for log10 targets."""
    if target in LOG_TARGETS:
        return np.power(10.0, y_t)
    return y_t


def check_files():
    missing = []
    if not os.path.exists(AUG_PATH):
        missing.append(AUG_PATH)
    for k, p in XGB_MODEL_FILES.items():
        if not os.path.exists(p):
            missing.append(p)
    if not os.path.exists(CONFORMAL_PATH):
        missing.append(CONFORMAL_PATH)

    if missing:
        print("Missing required files:")
        for m in missing:
            print(" ", m)
        raise FileNotFoundError("Upload missing Chapter 6/7 artifacts to /content.")


def conservative_gate(pred_row: Dict[str, Dict[str, float]]) -> Dict[str, Any]:
    """Decision rule using uncertainty-aware upper bounds."""
    keff_ub = pred_row["keff_est"]["upper"]
    pow_ub = pred_row["total_power_W"]["upper"]
    pd_ub = pred_row["avg_core_power_density_W_cm3"]["upper"]

    ok_keff = bool(keff_ub < KEFF_MAX)
    ok_pow = bool(pow_ub <= POWER_MAX_W)
    ok_pd = bool(pd_ub <= PD_MAX_W_CM3)

    status = "SAFE" if (ok_keff and ok_pow and ok_pd) else "REJECT"

    # Margin-to-envelope using upper bounds (conservative)
    margins = {
        "keff_margin_to_max": float(KEFF_MAX - keff_ub),
        "power_margin_to_max_W": float(POWER_MAX_W - pow_ub),
        "pd_margin_to_max_W_cm3": float(PD_MAX_W_CM3 - pd_ub),
    }

    return {
        "ok_keff": ok_keff,
        "ok_power": ok_pow,
        "ok_power_density": ok_pd,
        "status": status,
        "margins": margins,
    }


def lhs_sample(n: int, bounds: Dict[str, Tuple[float, float]], seed: int = 1234) -> np.ndarray:
    """Simple LHS in [0,1] then scale to bounds."""
    rng = np.random.default_rng(seed)
    d = len(bounds)

    cut = np.linspace(0, 1, n + 1)
    u = rng.random((n, d))
    a = cut[:n]
    b = cut[1:n+1]
    rdpoints = u * (b - a)[:, None] + a[:, None]

    H = np.zeros_like(rdpoints)
    for j in range(d):
        order = rng.permutation(n)
        H[:, j] = rdpoints[order, j]

    Xs = np.zeros((n, d), dtype=float)
    for j, key in enumerate(bounds.keys()):
        lo, hi = bounds[key]
        Xs[:, j] = lo + H[:, j] * (hi - lo)

    return Xs


def to_scenario_df(scenarios: Dict[str, List[float]]) -> pd.DataFrame:
    rows = []
    for name, vals in scenarios.items():
        if len(vals) != len(X_COLS):
            raise ValueError(f"Scenario {name} needs {len(X_COLS)} values.")
        r = {"scenario": name}
        for j, c in enumerate(X_COLS):
            r[c] = float(vals[j])
        rows.append(r)
    return pd.DataFrame(rows)

def predict_with_intervals(X_in: np.ndarray) -> Dict[str, Dict[str, np.ndarray]]:
    """Return per-target arrays: mean, lower, upper, q."""
    out = {}
    for tgt in Y_COLS:
        # Model predicts in transformed space for LOG_TARGETS.
        y_t = xgb_models[tgt].predict(X_in)
        y = inverse_transform_y(y_t, tgt)

        # Conformal q is stored in physical space (as computed in Chapter 7).
        q = float(res_q[tgt])

        out[tgt] = {
            "mean": y.astype(float),
            "lower": (y - q).astype(float),
            "upper": (y + q).astype(float),
            "q": np.full_like(y, q, dtype=float),
        }
    return out

# Block 5 — Load Artifacts
check_files()

print("Loading augmented dataset:")
df = pd.read_parquet(AUG_PATH)
print("  shape:", df.shape)

# Ensure numeric
for c in X_COLS + Y_COLS:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

SYN_FLAG_COL = "is_synthetic" if "is_synthetic" in df.columns else None

# Use real-only rows to define domain + coverage checks
if SYN_FLAG_COL:
    df_real = df[df[SYN_FLAG_COL] == False].copy()
    df_syn = df[df[SYN_FLAG_COL] == True].copy()
else:
    df_real = df.copy()
    df_syn = df.iloc[0:0].copy()

# Drop NaNs in required columns
need_cols = [c for c in X_COLS + Y_COLS if c in df_real.columns]
df_real = df_real.dropna(subset=need_cols).reset_index(drop=True)

print("  real rows:", len(df_real), " synthetic rows:", len(df_syn))

# Domain bounds from real data
DOMAIN = {c: (float(df_real[c].min()), float(df_real[c].max())) for c in X_COLS}
print("\nDerived design-domain bounds (real-only):")
for k, (a, b) in DOMAIN.items():
    print(f"  {k}: [{a:.6g}, {b:.6g}]")

# Output sanity bounds from real-only labels
# If ICS predicts outside these bounds, treat it as an invalid region for thesis-grade reporting.
Y_BOUNDS = {}
for y in Y_COLS:
    if y in df_real.columns:
        y_min = float(df_real[y].min())
        y_max = float(df_real[y].max())
        Y_BOUNDS[y] = (y_min, y_max)

# Small tolerance
Y_TOL = {
    "keff_est": 0.02,
    "flux_shape_metric": 0.05,
    "breeding_proxy_norm": 0.0,  # do not allow negative
    "total_power_W": 0.0,        # do not allow negative
    "avg_core_power_density_W_cm3": 0.0,
}

# Training geometry for distance diagnostics (standardized space)
X_real = df_real[X_COLS].to_numpy(dtype=float)

x_nn_scaler = StandardScaler()
X_real_s = x_nn_scaler.fit_transform(X_real)

centroid_s = X_real_s.mean(axis=0)

nn_real = NearestNeighbors(n_neighbors=1)
nn_real.fit(X_real_s)

# Optional: derive an OOD distance threshold from real-only neighbor distances
# Use 2-NN on real set to approximate "distance to nearest other real point"
nn_real_2 = NearestNeighbors(n_neighbors=2)
nn_real_2.fit(X_real_s)
d2, _ = nn_real_2.kneighbors(X_real_s)
# d2[:,0] is self-distance (0). d2[:,1] is nearest other point.
OOD_DIST_THRESH = float(np.quantile(d2[:, 1], 0.95))

# Load XGBoost models
xgb_models = {tgt: joblib.load(path) for tgt, path in XGB_MODEL_FILES.items()}

# Load conformal residual quantiles (physical space)
conf = load_json(CONFORMAL_PATH)
alpha = float(conf.get("alpha", 0.10))
res_q = conf.get("residual_quantile", {})

# Basic checks
for tgt in Y_COLS:
    if tgt not in res_q:
        raise ValueError(f"Missing conformal quantile for target: {tgt}")

print("\nLoaded ICS artifacts:")
print("  models:", list(xgb_models.keys()))
print("  alpha:", alpha)

# Block 6 — Scenario Definitions (data-driven, inside-domain, conservatively SAFE)
# We derive scenarios from a candidate scan so that Chapter 8 tables include SAFE and boundary behaviors.

def in_domain_mask(X_in: np.ndarray) -> np.ndarray:
    m = np.ones((X_in.shape[0],), dtype=bool)
    for j, c in enumerate(X_COLS):
        lo, hi = DOMAIN[c]
        m &= (X_in[:, j] >= lo) & (X_in[:, j] <= hi)
    return m

def output_sanity_ok(pred_row: Dict[str, Dict[str, float]]) -> Tuple[bool, Dict[str, Any]]:
    """
    If predicted mean is far outside the real-only output bounds, flag as invalid region.
    """
    flags = {}
    ok = True
    for tgt in Y_COLS:
        y = float(pred_row[tgt]["mean"])
        if tgt not in Y_BOUNDS:
            continue
        lo, hi = Y_BOUNDS[tgt]
        tol = float(Y_TOL.get(tgt, 0.0))
        # enforce non-negative where appropriate
        if tgt in {"breeding_proxy_norm", "total_power_W", "avg_core_power_density_W_cm3"} and y < 0.0:
            ok = False
            flags[tgt] = {"status": "NEGATIVE_PRED", "mean": y, "bounds": [lo, hi]}
            continue
        if y < (lo - tol) or y > (hi + tol):
            ok = False
            flags[tgt] = {"status": "OUTSIDE_REAL_BOUNDS", "mean": y, "bounds": [lo, hi], "tol": tol}
    return ok, flags

def scaled_nn_distance(x_vec: np.ndarray) -> float:
    xs = x_nn_scaler.transform(x_vec)
    d_nn, _ = nn_real.kneighbors(xs)
    return float(d_nn[0, 0])

# Candidate pool for scenario construction
N_SC_POOL = 80000
X_pool = lhs_sample(N_SC_POOL, DOMAIN, seed=SEED + 555)

# Predict pool
pred_pool = predict_with_intervals(X_pool)

# Conservative pass mask
keff_ub_pool = pred_pool["keff_est"]["upper"]
pow_ub_pool  = pred_pool["total_power_W"]["upper"]
pd_ub_pool   = pred_pool["avg_core_power_density_W_cm3"]["upper"]
pass_mask_pool = (keff_ub_pool < KEFF_MAX) & (pow_ub_pool <= POWER_MAX_W) & (pd_ub_pool <= PD_MAX_W_CM3)

idx_pass = np.where(pass_mask_pool)[0]
if len(idx_pass) < 50:
    raise RuntimeError("Too few SAFE candidates in scenario pool. Increase N_SC_POOL or check ICS artifacts.")

# Baseline SAFE: interior, informative subcritical regime
keff_mean_pool = pred_pool["keff_est"]["mean"][idx_pass]

# Define engineering-meaningful baseline band
KEFF_BASELINE_MIN = 0.75
KEFF_BASELINE_MAX = 0.90

baseline_mask = (
    (keff_mean_pool >= KEFF_BASELINE_MIN) &
    (keff_mean_pool <= KEFF_BASELINE_MAX)
)

if np.sum(baseline_mask) < 10:
    raise RuntimeError("Insufficient baseline candidates in keff interior band.")

# Among interior points, choose closest to real centroid
candidate_idx = idx_pass[baseline_mask]

dists = []
for ii in candidate_idx:
    x_vec = X_pool[ii:ii+1]
    x_vec_s = x_nn_scaler.transform(x_vec)
    dists.append(np.linalg.norm(x_vec_s.reshape(-1) - centroid_s))

ii_baseline = candidate_idx[int(np.argmin(dists))]

# Breeding oriented SAFE: maximize breeding mean among passing
breed_mean = pred_pool["breeding_proxy_norm"]["mean"][idx_pass]
ii_breeding = idx_pass[int(np.argmax(breed_mean))]

# Power oriented SAFE: maximize power mean among passing
power_mean = pred_pool["total_power_W"]["mean"][idx_pass]
ii_power = idx_pass[int(np.argmax(power_mean))]

# Boundary stress:
# Choose point with keff upper bound closest to KEFF_MAX but still SAFE,
# plus one near-miss (optional) that violates only because upper bound crosses KEFF_MAX.
ii_boundary_safe = idx_pass[int(np.argmax(keff_ub_pool[idx_pass]))]

# Near-miss candidate: smallest (keff_ub - KEFF_MAX) among violations
viol_mask = (keff_ub_pool >= KEFF_MAX)
idx_viol = np.where(viol_mask)[0]
ii_boundary_reject = None
if len(idx_viol) > 0:
    delta = keff_ub_pool[idx_viol] - KEFF_MAX
    ii_boundary_reject = idx_viol[int(np.argmin(delta))]

# Build scenarios dict using effective values that will actually be evaluated
scenarios = {
    "Baseline_SAFE": X_pool[ii_baseline].tolist(),
    "Breeding_SAFE": X_pool[ii_breeding].tolist(),
    "Power_SAFE": X_pool[ii_power].tolist(),
    "Boundary_SAFE": X_pool[ii_boundary_safe].tolist(),
}
if ii_boundary_reject is not None:
    scenarios["Boundary_NEAR_MISS"] = X_pool[ii_boundary_reject].tolist()

sc_df = to_scenario_df(scenarios)
X_sc = sc_df[X_COLS].to_numpy(dtype=float)

# Save effective scenarios (not the earlier hand-written values)
scenarios_path = os.path.join(OUT_DIR, "chapter8_scenarios.json")
with open(scenarios_path, "w", encoding="utf-8") as f:
    json.dump(scenarios, f, indent=2)

print("\nSaved scenarios (effective evaluated values):")
print(" ", scenarios_path)
print(sc_df.to_string(index=False))

# Block 7 — ICS Inference with Conformal Intervals


pred_sc = predict_with_intervals(X_sc)

# Assemble scenario table
rows = []
for i in range(len(sc_df)):
    name = sc_df.loc[i, "scenario"]
    xrow = sc_df.loc[i, X_COLS].to_dict()

    # Build per-scenario prediction dict
    pred_row = {}
    for tgt in Y_COLS:
        pred_row[tgt] = {
            "mean": float(pred_sc[tgt]["mean"][i]),
            "lower": float(pred_sc[tgt]["lower"][i]),
            "upper": float(pred_sc[tgt]["upper"][i]),
            "q": float(pred_sc[tgt]["q"][i]),
        }

    gate = conservative_gate(pred_row)

    # Distance diagnostics (standardized)
    x_vec = np.array([[xrow[c] for c in X_COLS]], dtype=float)
    x_vec_s = x_nn_scaler.transform(x_vec)
    d_nn, _ = nn_real.kneighbors(x_vec_s)
    d_nn = float(d_nn[0, 0])
    d_cent = float(np.linalg.norm(x_vec_s.reshape(-1) - centroid_s))
    ood_flag = bool(d_nn > OOD_DIST_THRESH)

    # Output sanity tagging
    sane_ok, sane_flags = output_sanity_ok(pred_row)

    # Combine status logic
    # Priority: invalid region > unsafe by constraints > safe
    status = gate["status"]
    if not sane_ok:
        status = "OOD_REJECT_OUTPUT"
    elif ood_flag and status == "SAFE":
        status = "SAFE_OOD_WARNING"

    flat = {
        "scenario": name,
        "status": status,
        "ok_keff": gate["ok_keff"],
        "ok_power": gate["ok_power"],
        "ok_power_density": gate["ok_power_density"],
        "ood_distance_scaled": float(d_nn),
        "ood_threshold_scaled_p95": float(OOD_DIST_THRESH),
        "ood_flag": bool(ood_flag),
        "dist_to_centroid_scaled": d_cent,
        "output_sanity_ok": bool(sane_ok),
        "output_sanity_flags": json.dumps(sane_flags) if len(sane_flags) else "",
    }

    # inputs
    for c in X_COLS:
        flat[c] = float(xrow[c])

    # outputs
    for tgt in Y_COLS:
        flat[f"{tgt}_mean"] = pred_row[tgt]["mean"]
        flat[f"{tgt}_lower"] = pred_row[tgt]["lower"]
        flat[f"{tgt}_upper"] = pred_row[tgt]["upper"]
        flat[f"{tgt}_q"] = pred_row[tgt]["q"]

    # margins
    flat["keff_margin_to_max"] = gate["margins"]["keff_margin_to_max"]
    flat["power_margin_to_max_W"] = gate["margins"]["power_margin_to_max_W"]
    flat["pd_margin_to_max_W_cm3"] = gate["margins"]["pd_margin_to_max_W_cm3"]

    rows.append(flat)

scenario_pred_df = pd.DataFrame(rows)

scenario_csv = os.path.join(OUT_DIR, "chapter8_scenario_predictions.csv")
scenario_pred_df.to_csv(scenario_csv, index=False)

print("\nScenario predictions saved:")
print(" ", scenario_csv)

print("\nScenario summary:")
print(scenario_pred_df[[
    "scenario", "status",
    "keff_est_mean", "keff_est_lower", "keff_est_upper",
    "total_power_W_mean", "total_power_W_upper",
    "avg_core_power_density_W_cm3_mean", "avg_core_power_density_W_cm3_upper",
    "ood_distance_scaled",
]].to_string(index=False))

# Block 8 — Bounded Candidate Search (ICS as an optimization assistant)
# We do not let the ICS define envelopes.
# We generate candidates inside DOMAIN, then filter by conservative constraints.
N_CAND = 60000
X_cand = lhs_sample(N_CAND, DOMAIN, seed=SEED + 888)

pred_cand = predict_with_intervals(X_cand)

# Build pass mask using upper bounds
keff_ub = pred_cand["keff_est"]["upper"]
pow_ub = pred_cand["total_power_W"]["upper"]
pd_ub = pred_cand["avg_core_power_density_W_cm3"]["upper"]

pass_mask = (keff_ub < KEFF_MAX) & (pow_ub <= POWER_MAX_W) & (pd_ub <= PD_MAX_W_CM3)

n_pass = int(np.sum(pass_mask))
print("\nCandidate scan:")
print("  candidates:", N_CAND)
print("  passing conservative gates:", n_pass)
print("  pass rate:", n_pass / max(1, N_CAND))

# Rank safe candidates by objectives
# Objective 1: maximize breeding_proxy_norm mean
# Objective 2: maximize total_power_W mean

rank_rows = []
if n_pass > 0:
    idx_pass = np.where(pass_mask)[0]

    breeding_mean = pred_cand["breeding_proxy_norm"]["mean"][idx_pass]
    power_mean = pred_cand["total_power_W"]["mean"][idx_pass]

    topN = 15

    # Top by breeding
    top_b = idx_pass[np.argsort(-breeding_mean)[:topN]]
    # Top by power
    top_p = idx_pass[np.argsort(-power_mean)[:topN]]

    def add_block(which: str, indices: np.ndarray):
        for r, ii in enumerate(indices, start=1):
            row = {"rank_type": which, "rank": int(r)}
            for j, c in enumerate(X_COLS):
                row[c] = float(X_cand[ii, j])
            for tgt in Y_COLS:
                row[f"{tgt}_mean"] = float(pred_cand[tgt]["mean"][ii])
                row[f"{tgt}_lower"] = float(pred_cand[tgt]["lower"][ii])
                row[f"{tgt}_upper"] = float(pred_cand[tgt]["upper"][ii])
                row[f"{tgt}_q"] = float(pred_cand[tgt]["q"][ii])
            # margins using upper bound
            row["keff_margin_to_max"] = float(KEFF_MAX - pred_cand["keff_est"]["upper"][ii])
            row["power_margin_to_max_W"] = float(POWER_MAX_W - pred_cand["total_power_W"]["upper"][ii])
            row["pd_margin_to_max_W_cm3"] = float(PD_MAX_W_CM3 - pred_cand["avg_core_power_density_W_cm3"]["upper"][ii])

            # distances
            x_vec = X_cand[ii:ii+1]
            d_nn, _ = nn_real.kneighbors(x_vec)
            row["dist_to_nearest_real"] = float(d_nn[0, 0])
            row["dist_to_centroid"] = float(np.linalg.norm(x_vec.reshape(-1) - centroid_s))
            rank_rows.append(row)

    add_block("top_breeding", top_b)
    add_block("top_power", top_p)

rank_df = pd.DataFrame(rank_rows)
rank_csv = os.path.join(OUT_DIR, "chapter8_candidate_rankings.csv")
rank_df.to_csv(rank_csv, index=False)

print("\nSaved candidate rankings:")
print(" ", rank_csv)

# Block 9 — Conformal Coverage Check (real-only held-out)
# We re-split real data to estimate empirical coverage for the conformal intervals.
# This is not recalibration. Calibration constants remain fixed.
# Use only real rows with valid columns
real_ok = df_real.dropna(subset=X_COLS + Y_COLS).reset_index(drop=True)
Xr = real_ok[X_COLS].to_numpy(dtype=float)
Yr = real_ok[Y_COLS].to_numpy(dtype=float)

# Small safety: if too few rows, skip
coverage_report = {
    "alpha": alpha,
    "nominal_coverage": float(1.0 - alpha),
    "n_real": int(len(real_ok)),
    "targets": {},
}

if len(real_ok) >= 200:
    idx_all = np.arange(len(real_ok))
    idx_cal, idx_eval = train_test_split(idx_all, test_size=0.25, random_state=SEED, shuffle=True)

    # Evaluate on idx_eval
    X_eval = Xr[idx_eval]
    Y_eval = Yr[idx_eval]

    pred_eval = predict_with_intervals(X_eval)

    for j, tgt in enumerate(Y_COLS):
        y_true = Y_eval[:, j]
        lo = pred_eval[tgt]["lower"]
        up = pred_eval[tgt]["upper"]

        covered = (y_true >= lo) & (y_true <= up)
        coverage = float(np.mean(covered))
        mean_width = float(np.mean(up - lo))

        coverage_report["targets"][tgt] = {
            "empirical_coverage": coverage,
            "mean_interval_width": mean_width,
            "q_used": float(res_q[tgt]),
        }

else:
    coverage_report["note"] = "Insufficient real rows for a stable coverage estimate."

coverage_path = os.path.join(OUT_DIR, "chapter8_coverage_report.json")
with open(coverage_path, "w", encoding="utf-8") as f:
    json.dump(coverage_report, f, indent=2)

print("\nCoverage report saved:")
print(" ", coverage_path)

# Block 10 — Plots (saved to disk)

def save_hist_real_vs_scenarios(col: str):
    plt.figure()
    r = df_real[col].dropna().to_numpy(dtype=float)
    plt.hist(r, bins=60, alpha=0.65, label="real")

    # scenario means
    s = scenario_pred_df[f"{col}_mean"].to_numpy(dtype=float)
    plt.scatter(s, np.full_like(s, 0.0), marker="x", s=80, label="scenarios")

    plt.xlabel(col)
    plt.ylabel("count")
    plt.title(f"Real distribution vs scenario means: {col}")
    plt.legend()
    out = os.path.join(FIG_DIR, f"hist_real_vs_scenarios_{col}.png")
    plt.savefig(out, dpi=180, bbox_inches="tight")
    plt.close()


def save_reject_rate_vs_ood_distance():
    # Use a sample of candidates to keep plot light
    n_plot = min(30000, len(X_cand))
    rng = np.random.default_rng(SEED + 99)
    idxp = rng.choice(len(X_cand), size=n_plot, replace=False)

    Xp = X_cand[idxp]
    Xp_s = x_nn_scaler.transform(Xp)
    d_nn, _ = nn_real.kneighbors(Xp_s)
    d_nn = d_nn[:, 0]

    pred_p = predict_with_intervals(Xp)

    keff_ub = pred_p["keff_est"]["upper"]
    pow_ub  = pred_p["total_power_W"]["upper"]
    pd_ub   = pred_p["avg_core_power_density_W_cm3"]["upper"]

    reject = ~((keff_ub < KEFF_MAX) & (pow_ub <= POWER_MAX_W) & (pd_ub <= PD_MAX_W_CM3))

    # Bin by OOD distance deciles
    qs = np.quantile(d_nn, np.linspace(0.0, 1.0, 11))
    bin_centers = []
    reject_rates = []

    for b in range(10):
        lo, hi = qs[b], qs[b+1]
        m = (d_nn >= lo) & (d_nn <= hi)
        if np.sum(m) < 50:
            continue
        bin_centers.append(0.5 * (lo + hi))
        reject_rates.append(float(np.mean(reject[m])))

    plt.figure()
    plt.plot(bin_centers, reject_rates, marker="o")
    plt.xlabel("OOD distance (scaled), binned deciles")
    plt.ylabel("reject fraction under conservative gates")
    plt.title("Robustness: reject rate increases with distance from real training support")
    out = os.path.join(FIG_DIR, "reject_rate_vs_ood_distance.png")
    plt.savefig(out, dpi=180, bbox_inches="tight")
    plt.close()

save_reject_rate_vs_ood_distance()

print("\nSaved figures to:")
print(" ", FIG_DIR)

# Block 11 — Chapter 8 Report JSON
report = {
    "purpose": "ICS evaluation and bounded optimization support under fixed human-defined safety envelopes",
    "envelopes": {
        "keff_max": KEFF_MAX,
        "power_max_W": POWER_MAX_W,
        "power_density_max_W_cm3": PD_MAX_W_CM3,
    },
    "domain_real_only": {k: [float(v[0]), float(v[1])] for k, v in DOMAIN.items()},
    "alpha": alpha,
    "nominal_coverage": float(1.0 - alpha),
    "scenario_predictions_csv": os.path.basename(scenario_csv),
    "candidate_rankings_csv": os.path.basename(os.path.join(OUT_DIR, "chapter8_candidate_rankings.csv")),
    "coverage_report_json": os.path.basename(coverage_path),
    "n_candidates_scanned": int(N_CAND),
    "n_candidates_passing": int(n_pass),
}

report_path = os.path.join(OUT_DIR, "chapter8_report.json")
with open(report_path, "w", encoding="utf-8") as f:
    json.dump(report, f, indent=2)

print("\nSaved Chapter 8 report:")
print(" ", report_path)

print("\nDONE: Chapter 8 evaluation pipeline complete.")

Loading augmented dataset:
  shape: (6841, 10)
  real rows: 1500  synthetic rows: 5341

Derived design-domain bounds (real-only):
  core_radius_cm: [50.0683, 149.954]
  blanket_thickness_cm: [0.164555, 99.954]
  height_cm: [100.093, 299.713]
  initial_U233_frac: [1.39817e-05, 0.0471365]

Loaded ICS artifacts:
  models: ['keff_est', 'flux_shape_metric', 'breeding_proxy_norm', 'total_power_W', 'avg_core_power_density_W_cm3']
  alpha: 0.1

Saved scenarios (effective evaluated values):
  Chapter8_Output/chapter8_scenarios.json
          scenario  core_radius_cm  blanket_thickness_cm  height_cm  initial_U233_frac
     Baseline_SAFE       91.933276             48.898336 209.661394           0.014113
     Breeding_SAFE       52.919143              3.984475 131.358085           0.011097
        Power_SAFE      113.127141             64.331227 288.148639           0.018691
     Boundary_SAFE       90.444503             42.532162 161.606369           0.022596
Boundary_NEAR_MISS       57.826431  